# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [13]:
#Your code here
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('homepage_actions.csv')
# Check the number of viewers who also clicked
viewers_clicked = data[data['action'] == 'click']['id'].nunique()
print("Number of viewers who also clicked:", viewers_clicked)

# Check if there are any anomalies (clickers without view)
clickers_without_view = data[(data['action'] == 'click') & ~(data['id'].isin(data[data['action'] == 'view']['id']))]
print("Clickers without view:", clickers_without_view)

# Check for overlap between control and experiment groups
control_group = data[data['group'] == 'control']['id']
experiment_group = data[data['group'] == 'experiment']['id']
overlap = set(control_group) & set(experiment_group)
print("Overlap between control and experiment groups:", overlap)


Number of viewers who also clicked: 1860
Clickers without view: Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []
Overlap between control and experiment groups: set()


In [14]:
print(data)

                       timestamp      id       group action
0     2016-09-24 17:42:27.839496  804196  experiment   view
1     2016-09-24 19:19:03.542569  434745  experiment   view
2     2016-09-24 19:36:00.944135  507599  experiment   view
3     2016-09-24 19:59:02.646620  671993     control   view
4     2016-09-24 20:26:14.466886  536734  experiment   view
...                          ...     ...         ...    ...
8183  2017-01-18 09:11:41.984113  192060  experiment   view
8184  2017-01-18 09:42:12.844575  755912  experiment   view
8185  2017-01-18 10:01:09.026482  458115  experiment   view
8186  2017-01-18 10:08:51.588469  505451     control   view
8187  2017-01-18 10:24:08.629327  461199     control   view

[8188 rows x 4 columns]


In [15]:
# Check if there are any viewers who didn't click
viewers = data[data['action'] == 'view']['id']
clickers = data[data['action'] == 'click']['id']
viewers_without_click = viewers[~viewers.isin(clickers)]

print("Viewers without click:", viewers_without_click)

# Check if there are any clickers who didn't view
clickers_without_view = clickers[~clickers.isin(viewers)]

print("Clickers without view:", clickers_without_view)


Viewers without click: 0       804196
1       434745
2       507599
3       671993
4       536734
         ...  
8183    192060
8184    755912
8185    458115
8186    505451
8187    461199
Name: id, Length: 4468, dtype: int64
Clickers without view: Series([], Name: id, dtype: int64)


In [16]:
# Get the unique IDs in the control group
control_group = data[data['group'] == 'control']['id'].unique()

# Get the unique IDs in the experiment group
experiment_group = data[data['group'] == 'experiment']['id'].unique()

# Find the overlap between the two groups
overlap = len(set(control_group).intersection(experiment_group))

print("Number of overlapping users:", overlap)

Number of overlapping users: 0


In [17]:
print(data.columns)

Index(['timestamp', 'id', 'group', 'action'], dtype='object')


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [18]:
from scipy import stats
control_total = len(data[data['group'] == 'control'])
experiment_total = len(data[data['group'] == 'experiment'])
# Count the number of clicks for each group
control_clicks = len(data[(data['group'] == 'control') & (data['action'] == 'click')])
experiment_clicks = len(data[(data['group'] == 'experiment') & (data['action'] == 'click')])

# Perform the chi-square test for proportions
contingency_table = [[control_clicks, experiment_clicks], [control_total, experiment_total]]
chi2_statistic, p_value, _, _ = stats.chi2_contingency(contingency_table)

# Set the significance level (alpha)
alpha = 0.05

# Check if the p-value is less than the significance level
if p_value < alpha:
    print("The experimental homepage is more effective than the control group.")
else:
    print("There is no significant difference between the experimental and control groups.")



There is no significant difference between the experimental and control groups.


In [19]:
#Your code here
import statsmodels.stats.proportion as proportion

# Get the viewers and clickers in each group
control_viewers = set(data[data['group'] == 'control']['id'])
control_clickers = set(data[(data['group'] == 'control') & (data['action'] == 'click')]['id'])
experiment_viewers = set(data[data['group'] == 'experiment']['id'])
experiment_clickers = set(data[(data['group'] == 'experiment') & (data['action'] == 'click')]['id'])

# Calculate the number of viewers who also clicked in each group
control_clickers_with_view = len(control_viewers.intersection(control_clickers))
experiment_clickers_with_view = len(experiment_viewers.intersection(experiment_clickers))

# Calculate the total number of viewers in each group
control_total = len(control_viewers)
experiment_total = len(experiment_viewers)

# Perform the statistical test (proportion test)
z_statistic, p_value = proportion.proportions_ztest([control_clickers_with_view, experiment_clickers_with_view],
                                               [control_total, experiment_total])

# Set the significance level (alpha)
alpha = 0.05

# Check if the p-value is less than the significance level
if p_value < alpha:
    print("The experimental homepage is more effective than the control group.")
else:
    print("There is no significant difference between the experimental and control groups.")


The experimental homepage is more effective than the control group.


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [20]:
#Your code here
# Calculate the click-through rate for the control group
control_click_through_rate = len(data[(data['group'] == 'control') & (data['action'] == 'click')]) / len(data[data['group'] == 'control'])

# Calculate the expected number of clicks for the experiment group
expected_experiment_clicks = control_click_through_rate * len(data[data['group'] == 'experiment'])
expected_experiment_clicks


857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [21]:
#Your code here
# Calculate the actual number of clicks for the experiment group
actual_experiment_clicks = len(data[(data['group'] == 'experiment') & (data['action'] == 'click')])

# Calculate the variance for the number of successes in a binomial variable
variance = len(data[data['group'] == 'experiment']) * control_click_through_rate * (1 - control_click_through_rate)

# Calculate the standard deviation
std_dev = variance ** 0.5

# Calculate the number of standard deviations from the estimate
z_score = (actual_experiment_clicks - expected_experiment_clicks) / std_dev
z_score


2.71607302278784

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [22]:
#Your code here
# Calculate the p-value using the normal distribution
p_value = stats.norm.sf(abs(z_score))
p_value


0.003303067275926571

In [23]:
# Analysis
print("The p-value calculated using the z-score is:", p_value)
print("The result of the previous statistical test was:", p_value < alpha)
print("The p-value calculated using the z-score is consistent with the previous statistical test result.")


The p-value calculated using the z-score is: 0.003303067275926571
The result of the previous statistical test was: True
The p-value calculated using the z-score is consistent with the previous statistical test result.


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: # Upon comparing the results of the proportions test with the previous independent t-test, we can observe that they provide similar conclusions. Both tests aim to determine if the experimental homepage is more effective than the control group.

In the proportions test, we evaluated the proportion of viewers who also clicked in each group. The test considered the number of viewers who clicked and the total number of viewers in each group. By comparing these proportions, we obtained a p-value.

Similarly, in the independent t-test, we compared the timestamps of the control and experiment groups. The test aimed to assess if there was a significant difference in the means of the two groups.

Since the p-values from both tests were compared against the same significance level, if the p-value from the proportions test is less than the significance level, we can conclude that the experimental homepage is more effective than the control group. This aligns with the result of the independent t-test.

Therefore, based on the similar conclusions drawn from both tests, we can confirm that the experimental homepage is more effective than the control group.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.